# Interpreter für Registermaschinen
Dieser Interpreter führt Registermaschinen-Programme aus.

Version 2.3

## Syntax für Registermaschinen-Programme
Es gibt folgende Befehle für Maschinenprogramme:

* `PUSH    ` _reg\_nr_ _symbol_ 
* `POP     ` _reg\_nr_ [_:marke\_leer_] {_char:marke_} [_else\_marke_]
* `SWITCH  ` _reg\_nr_ [_:marke\_leer_] {_char:marke_} [_else\_marke_]
* `GOTO    ` _marke_
* `STOP    `
* `SUB     ` _Name\_eines\_Sub-Programmes_ {_parameter\_name_}] 
* `CALL    ` _Name\_eines\_Sub-Programmes_  {_parameter\_wert_}] 

Parameternamen beginnen mit einem '$' und besitzen eine Länge \>= 2.  
In dieser Version werden Parameter nur für Register-Nummern unterstützt,

Jedem Befehl kann eine Marke vorangestellt werden: _marke:_        
Vor dem Doppelpunkt der Zeilenmarken darf kein Leerraum stehen, nach dem Doppelpunkt muss ein Lehrraum stehen. 

Die Registermaschine kann um weitere Befehle erweitert werden.
(TODO: Doku dazu)

### Bemerkungen zur Syntax
1. Alle Befehle sind vom gleichen Schema:     
`Befehlsname` _Parameter1 Parameter2 etc._
2. Zum `SWITCH`-Befehl. Beispiel:       
     `SWITCH reg_nr :marke_empty  a:marke_a   b:marke_b  ...  else_marke`    
     Die Marken sind optional.     
     Es wird ein Sprung ausgeführt, abhängig davon, was das oberste Symbol im Register `reg_nr` ist:
    + ist das Register leer, so wird zu `marke_empty` gesprungen.
    + ist das oberste Symbol des Registers ein `a` oder ein `b`, so wird nach `marke_a` oder `marke_b` gesprungen.
    + Anderenfalls wird zu `else_marke` gesprungen. Fehlt die `else_marke`, so wird einfach zur nächsten Anweisung gegangen.
3. Der `POP`-Befehl macht im Wesentlichen das gleiche wie der `SWITCH`-Befehl, entfernt aber das oberste Symbol aus dem Register.    
   Bei einem `POP`-Befehl ohne Sprung-Marken wird einfach das oberste Symbol aus dem Register entfernt und zum nächsten Befehl übergegangen.
  
## API
+ Eine Registermaschine wird erstellt mit      
`rm = Registermaschine("""` _Programm-Quellcode_ `""")`.
+ Eine solche Registermaschine ist «Callable», kann also aufgerufen werden:      
`rm(r0, r1, ... , rn)`.    
Der Rückgabewert ist der Inhalt aller Register, die verwendet worden sind.
+ Mit `rm.last_run()` wird ein Dataframe erstellt, das den gesamten Verlauf eines Runs anzeigt.      
    + Mit dem Parameter `align_left=True` wird erzwungen, dass die Zellen linksbündig ausgerichtet sind im Display; allerdings ist das Resultat dann kein Dataframe mehr, sondern ein Style-Objekt.    
    + Mit `align_left=False` wird ein Dataframe ausgegeben; Default ist `align_left=True`.
+ Die Anzahl Schritte, die ausgeführt werden, ist defaultmäßig auf 1000 beschränkt. Diese Schranke kann (global) angepasst werden durch `Registermaschine.MAX_STEPS = ??`.
      
(Für diese Methode sollte Pandas installiert sein.) 

### Feedback
+ Fehler bitte melden an urs-martin.kuenzi@ffhs.ch

## Beispiele
Beispiele aus dem Skript

In [ ]:
from Registermaschine import Registermaschine

In [ ]:
# Löschprogramm
# Lösche Inhalt von Register 2
# Alphabet Gamma beliebig
loeschen_R2_src = """
st:  POP 2 :end st
end: STOP
"""
loeschen_R2 = Registermaschine(loeschen_R2_src)
loeschen_R2("a", "ab", "abac")

In [ ]:
loeschen_R2.last_run()

In [ ]:
# Spiegelprogramm
# Der Wert des 1-ten Registers wird in das 0-te Register gespiegelt
spiegeln_src = """
st:  POP  1  :end  a:m_a  b:m_b
m_a: PUSH 0 a
     GOTO st
m_b: PUSH 0 b
     GOTO st
end: STOP
"""
spiegeln = Registermaschine(spiegeln_src)
spiegeln("", "aabbb")

In [ ]:
spiegeln.last_run()

In [ ]:
# Kopierprogramm:
# Der Wert des 1-ten Registers wird in das 0-te Register kopiert
# Das 2-te Register wird als Hilfsregister verwendet
# Gamma = {a, b}

copy_src = """
start:   POP 1 :zurueck a:mv_a b:mv_b
mv_a:    PUSH 2 a
         GOTO start
mv_b:    PUSH 2 b
         GOTO start
zurueck: POP 2 :end a:mv__a b:mv__b
mv__a:   PUSH 0 a
         PUSH 1 a
         GOTO zurueck
mv__b:   PUSH 0 b
         PUSH 1 b
         GOTO zurueck
end:     STOP
"""

copy = Registermaschine(copy_src)
copy("", "aaab")

In [ ]:
copy.last_run()

In [ ]:
# Kopierprogramm mit Subprogramm:
# Der Wert des 1-ten Registers wird in das 0-te Register kopiert
# Das 2-te Register wird als Hilfsregister verwendet
# Gamma = {a, b}

copy2_src = """
CALL spiegeln  1  2
CALL spiegeln  2  0

SUB spiegeln $ausgang $ziel 
start:   POP $ausgang :end a:mv_a b:mv_b
mv_a:    PUSH $ziel a
         GOTO start
mv_b:    PUSH $ziel b
         GOTO start
end:     STOP
"""

copy2 = Registermaschine(copy2_src)
copy2("", "aaab")

In [ ]:
copy2.last_run()

In [ ]:
# Programm zur Addition von 1 zu einer Binärzahl
# Die Zahl steht bei der Eingabe im ersten Register
# Das Resultat steht im 0-ten Register
# Gamma = {0, 1}

plus_1_src = """
# ue1: Zustand mit Übertrag 1
# ue1: Zustand mit Übertrag 0
ue1:      POP 1 :ue1_ 0:ue10 1:ue11
ue0:      POP 1 :ue0_ 0:ue00 1:ue01
ue0_:     GOTO spiegeln
ue00:     PUSH 2 0
          GOTO ue0
ue01:     PUSH 2 1
          GOTO ue0
ue1_:     PUSH 2 1
          GOTO spiegeln
ue10:     PUSH 2 1
          GOTO ue0
ue11:     PUSH 2 0
          GOTO ue1
spiegeln: POP 2 :end 0:mv_0 1:mv_1
mv_0:     PUSH 0 0
          GOTO spiegeln
mv_1:     PUSH 0 1
          GOTO spiegeln
end:      STOP
"""

plus_1 = Registermaschine(plus_1_src)
print(plus_1("", "10011"))
print(plus_1("", "111"))